# Convert Ehost annotation to MedSpacy Document Object medspacy-io-0.1.0.dev34
In this note book, we read in annotations and build SpanGroup from overlapped spans. We also build dataframe from annotation for building up OHDSI format database later.

In [ ]:
import os, medspacy_io
from pathlib import Path
doc_file = '../Development_and_Testing/mimic_annotations/testEhostReaderData/corpus/18305.txt' #replace with .txt
anno_file = '../Development_and_Testing/mimic_annotations/testEhostReaderData/saved/18305.txt.knowtator.xml' #replace with .xml
schema_file = '../Development_and_Testing/mimic_annotations/testEhostReaderData/config/projectschema.xml' #replace with .xml for schema

In [ ]:
print(Path(doc_file).read_text())


In [ ]:
print(Path(anno_file).read_text())

# eHost Doc Reader

In [ ]:
from spacy.lang.en import English
from medspacy_io.reader import EhostDocReader
from medspacy_io.reader import EhostDirReader

## Remark:
1. if you are using `support_overlap=True` the spans/annotations will end up in `doc._.concepts` , if you are using `support_overlap=False` they're in `doc.ents`. In our case it is `support_overlap=True`.
2. `EhostDocReader` will search default folder according to the directory of data. The data and annotations have to be organized as eHost Project:
    1. schema file: `../config/projectsschema.xml`
    2. data folder: `../corpus/*.txt`
    3. annotation folder: `../saved/*.txt.knowtator.xml`

In [ ]:
ereader = EhostDocReader(nlp=English(), schema_file=schema_file,support_overlap=True)
print(schema_file,doc_file)
doc = ereader.read(doc_file)
doc._.concepts

In [ ]:
doc_concepts_dic = doc._.concepts # doc is doc object readin by eHostReader
print(doc_concepts_dic)
annot_classes = doc_concepts_dic.keys()
print(annot_classes)
print(len(doc_concepts_dic['Symptom_Section']))


#doc.text
#re.finditer(testText,doc.text)
#for match in re.finditer(doc.text, doc_concepts_dic['Symptom_Section'][0]):
 #   print(match.start(),match.end())



In [ ]:
#doc.spans(doc_concepts_dic['Symptom_Section'])
testSpan = doc_concepts_dic['Symptom_Section'][1]
type(testSpan)
print(testSpan.text)
print(testSpan.label_)
print(testSpan.has_extension)
print(testSpan.start_char, testSpan.end_char)

# Build SpanGroup

In [ ]:
from spacy.tokens import SpanGroup
#doc.spans["errors"] = spans
group = SpanGroup(doc, name="Symptom_Section", spans=doc_concepts_dic['Symptom_Section'])
doc.spans["Symptom_Section"] = group


In [ ]:
type(doc_concepts_dic['Symptom_Section'][0])
type(doc.spans["Symptom_Section"])
print(len(doc.spans["Symptom_Section"]))
assert doc.spans["Symptom_Section"][0]==doc_concepts_dic['Symptom_Section'][0] #the first element in spanGroup is the first element in span list
print(doc.spans["Symptom_Section"].has_overlap,doc.spans["Symptom_Section"].__len__)
#doc_concepts_dic['Symptom_Section'][0].span_txt

# Build OHDSI format database from annotations

This is based on https://github.com/OHDSI/CommonDataModel/blob/main/inst/ddl/5.4/sql_server/OMOPCDM_sql_server_5.4_ddl.sql

In [ ]:
for concept,listSpan in doc_concepts_dic.items():
    print(concept, len(listSpan),listSpan)

In [ ]:
import pandas as pd
data = []
for concept, listSpan in doc_concepts_dic.items():
    for sp in listSpan:
        dicElem = {}
        dicElem['Span Text'] = sp.text
        dicElem['Concept Label'] = sp.label_
        dicElem['start loc'] = sp.start_char
        dicElem['end loc'] = sp.end_char
        dicElem['doc name'] = '18305.txt'
        
    data.append(dicElem)
        
   
print(data)

In [ ]:
df = pd.DataFrame(data)
df